# Dealing with Bias and Fairness in AI/Data Science Systems
## AAAI 2021 Hands-on Tutorial
### Pedro Saleiro, Kit Rodolfa, Rayid Ghani

# <font color=red>Exploring Bias Reduction Strategies</font>

## 1. Install dependencies, import packages and data
This is needed every time you open this notebook in **colab** to install dependencies

In [ ]:
# packages not installed in colab
!pip install aequitas==0.42.0
!pip install fairlearn==0.4.6
!pip install hyperparameter-tuning
!pip install fairgbm==0.9.14

import yaml
import os
import math
import pandas as pd
import numpy as np
import seaborn as sns
from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.fairness import Fairness
import aequitas.plot as ap
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import fairlearn
import warnings
warnings.filterwarnings("ignore")
sns.set() 
DPI = 200
DATAPATH = 'https://github.com/dssg/fairness_tutorial/raw/master/data/'

## What has already happened?

We've already cleaned data, generated features, created train-test sets, built 1000s of models on each training set and scored each test set with them, and calculated various evaluation metrics. 

As described earlier, the goal here is to select top 1000 project submissions that are likely to not get funded in order to prioritize resource allocation. That corresponds to the metric **Precision at top 1000**.

### <font color=red>We audited the "best" model at precision at top 1000 and found that it has disparities for True Positive Rate for all attributes that we care about (poverty_level of the school, sex, and school_location_type)</font>


## What do we want to do now?

1. Could we have picked a different model that was similar enough in "precision at top 1000" but less biased?


## Load predictions, labels, and attributes for all models that were built to audit

We have trained 400 models using random sampling of hyperparameters for the following algorithms: RandomForest, Logistic Regression, MLP and LightGBM. The `evals_df` contains a table with the performance metrics for each model on the holdout test set.

In [ ]:
evals_df = pd.read_csv(DATAPATH +'split2_evals.csv.gz', compression='gzip')
# Let's sort the models by Precision at top 1000 predicted positives (our performance metric of interest for this case study)
evals_df.sort_values('model_precision', ascending = False)
evals_df.head(10)

### Spot-check the model with highest precision at 1000 to see what type it is

In [ ]:
print('Highest precision model classpath and hyperparameters: \n\n' , evals_df['model_classpath'][0], '\n', evals_df['hyperparameters'][0])

## Load pre-computed Aequitas audit results 

The `aequitas_df` contains a table with the bias audit results for the 400 models for all the attributes we care about (we pre-selected the metrics of interest for this tutorial). 

In [ ]:
aequitas_df = pd.read_csv(DATAPATH + 'split2_aequitas.csv.gz', compression='gzip')
aequitas_df.head(10)

Combining the `evals_df` with the `aequitas_df` allow us to get a clearer picture how the different models compare with each other in both performance and bias of our groups of interest. 

The `create_scatter_disparity_performance` method will help us to easily plot the 400 models bias-performance tradeoffs and it's going to be used throughout the notebook.

In [ ]:
def create_scatter_disparity_performance(evals_df, aequitas_df,  attr_col, group_name, 
                                         performance_col='model_precision', bias_metric='tpr', flip_disparity=False, 
                                         mitigated_tags=[], mitigated_bdfs=[], mitigated_performances=[], ylim=None):
    disparity_df = aequitas_df.loc[(aequitas_df['attribute_name']==attr_col) & (aequitas_df['attribute_value']==group_name)].copy()
    disparity_metric = bias_metric + '_disparity'
    scatter_schema = ['model_uuid', performance_col, 'attribute_name', 'attribute_value', bias_metric, disparity_metric, 'model_tag']
    if flip_disparity:
        disparity_df[disparity_metric]= disparity_df.apply(lambda x: 1/x[disparity_metric] , axis=1)
    scatter = pd.merge(evals_df, disparity_df, how='left', on=['model_uuid'], sort=True, copy=True)
    scatter = scatter[['model_uuid', performance_col, 'attribute_name', 'attribute_value', bias_metric, disparity_metric]].copy()
    scatter['model_tag'] = 'Other Models'
    scatter.sort_values('model_precision', ascending = False, inplace=True, ignore_index=True)
    scatter['model_tag'] = scatter.apply(lambda x: 'Highest Precision at 1000' if int(x.name) < 1 else x['model_tag'], axis=1)
    mitigated_points = []
    scatter_final = pd.DataFrame()
    if mitigated_bdfs and mitigated_performances:
        for i in range(len(mitigated_bdfs)):
            if not mitigated_bdfs[i].empty and mitigated_performances[i] !=None:
                mitigated_bdfs[i][performance_col] = mitigated_performances[i]
                mitigated_bdfs[i]['model_tag'] = mitigated_tags[i]
                new_disparity_df = mitigated_bdfs[i].loc[(mitigated_bdfs[i]['attribute_name']==attr_col) & (mitigated_bdfs[i]['attribute_value']==group_name)].copy()
                if flip_disparity:
                    new_disparity_df[disparity_metric]= new_disparity_df.apply(lambda x: 1/x[disparity_metric] , axis=1)
                scatter_new = new_disparity_df[[c for c in new_disparity_df.columns if c in scatter_schema]].copy()
                mitigated_points.append(scatter_new)
        scatter_final = pd.concat(mitigated_points , axis=0)

    ax = sns.scatterplot(
        x='model_precision', y=disparity_metric, hue='model_tag',
        data=scatter,
        alpha=0.4, s=20, palette = np.array(sns.color_palette()[0:2])


    )
    if not scatter_final.empty:
        ax1 = sns.scatterplot(
            x='model_precision', y=disparity_metric, hue='model_tag',
            data=scatter_final,
            alpha = 0.95, s=20,  palette = np.array(sns.color_palette()[2:3] + sns.color_palette()[4:])
        )

    if ylim:
        plt.ylim(0, 10)
    flip_placeholder = 'Flipped' if flip_disparity else ''
    ax.set_title('{} {} vs.{} for {}:{}'.format(flip_placeholder, disparity_metric, performance_col, attr_col,group_name ), y=1., fontsize='xx-small')
    #plt.gcf().set_size_inches((4, 3))
    plt.legend(loc='upper left', fontsize='xx-small', prop={'size': 5})
    plt.gcf().set_dpi(DPI)
    plt.show() 




## Let's see if we could have picked a better model for fairness in Poverty Level

Each point in the scatterplot represents a model. We highlight the model we picked before (highest global performance).

In [ ]:
create_scatter_disparity_performance(evals_df, aequitas_df, 'poverty_level', 'highest', flip_disparity=True )

## Let's see if we could have picked a better model for fairness in Metro Type


In [ ]:
create_scatter_disparity_performance(evals_df, aequitas_df, 'metro_type', 'urban', flip_disparity=True, ylim=10 )

## Let's see if we could have picked a better model for fairness in Teacher sex



In [ ]:
create_scatter_disparity_performance(evals_df, aequitas_df, 'teacher_sex', 'female', flip_disparity=False )

# Bias Reduction Strategies
We will try a few different bias reduction strategies now and compare the audit results with the original models

1. Re-Sampling
2. Regularization (using Fairlearn package)
3. Post-hoc adjustment


## <font color="red">Bias Reduction Strategy 1: Re-Sampling</font>
### Can resampling approaches help improve the fairness of our models?

1. Load data
2. Look at training data distributions
3. Try Resampling in a few different ways
4. Rebuild model(s) on resampled training data
5. Predict on the test set
6. Audit for Bias and Compare

### What has already happened?

We've already cleaned data, generated features, created train-test sets, built 1000s of models on each training set and scored each test set with them, and calculated various evaluation metrics. We then used these results to pick a "best" model in terms of performance on the "accuracy" metric we care about: **Precision at the top 1000** (corresponding to our goal of selecting 1000 project submissions that are most likely to not get funded in order to prioritize resource allocation).

When we audited this selected model with Aequitas, however, we found biases across many attributes, including the poverty level of the schools. Here, we explore a methods based on resampling to reduce this bias in the selected model.

### Load data

In [ ]:
traindf = pd.read_csv(DATAPATH + 'train_20120501_20120801.csv.gz', compression='gzip')
testdf = pd.read_csv(DATAPATH + 'test_20121201_20130201.csv.gz', compression='gzip')
train_attrdf = pd.read_csv(DATAPATH + 'train_20120501_20120801_protected.csv.gz', compression='gzip')
test_attrdf = pd.read_csv(DATAPATH + 'test_20121201_20130201_protected.csv.gz', compression='gzip')


In [ ]:
traindf.shape

### Load pre-built models and predictions

The `evals_df` contains a table with the performance metrics for each model on the holdout test set.

In [ ]:
evals_df = pd.read_csv(DATAPATH +'split2_evals.csv.gz', compression='gzip')
# Let's sort the models by Precision at top 1000 predicted positives (our performance metric of interest for this case study)
evals_df.sort_values('model_precision', ascending = False)

### Take a look at the "Best" performing model

In [ ]:
print('Highest precision model classpath and hyperparameters: \n\n' , evals_df['model_classpath'][0], '\n', evals_df['hyperparameters'][0])

In [ ]:
#Let's load the hyperparameters of the highest performance model to a dictionary
import ast
hyperparameters= ast.literal_eval(evals_df['hyperparameters'][0])

In [ ]:
# Since the model with highest performance is a RandomForestClassifier let's create a classifier with the fetched hyperparameters
rf = RandomForestClassifier(**hyperparameters)

In [ ]:
rf.__dict__

### Defining label, group of interest and reference group for the attribute column we care about


In [ ]:
label_col = 'quickstart_label'
attribute_col = 'poverty_level'

group_of_interest = 'highest'
reference_group = 'lower'

### Look at training data distributions

#### Poverty_level=Highest
label_pos_poverty_highest =  P(poverty_level=highest | not_funded)

label_neg_poverty_highest =  P(poverty_level=highest | funded)


#### Poverty_level=Lower
label_pos_poverty_lower =  P(poverty_level=lower | not_funded)

label_neg_poverty_lower =  P(poverty_level=lower | funded)


In [ ]:
train_attrdf[attribute_col].value_counts()

In [ ]:
def get_group_stats(df, attrdf, label_col, attribute_col, group_name):
    """Calculates group-wise stats: prevalence, group_size, num label pos, num label neg
       Returns: prevalence, group_size, df of group label pos, df of group label neg
    """
    group_size = len(df.loc[attrdf[attribute_col]==group_name])
    label_pos = df.loc[(attrdf[attribute_col]==group_name) & (df[label_col] > 0)].copy()
    label_neg = df.loc[(attrdf[attribute_col]==group_name) & (df[label_col] < 1.0)].copy()
    assert (group_size == len(label_pos) + len(label_neg)), "label pos + label neg is different from group size!"
    prev = len(label_pos) / group_size
    print("{} == '{}'\n\t group size: {} \n\t prevalence: {:.3f}".format(attribute_col,group_name, group_size, prev))
    return prev, group_size, label_pos, label_neg

In [ ]:
_ = get_group_stats(traindf, train_attrdf, label_col, attribute_col, group_of_interest)

In [ ]:
_ = get_group_stats(traindf, train_attrdf, label_col, attribute_col, reference_group)

### What type of disparities do we see in the data distribution here?


1.

2.

3.

4.


















...







In [ ]:
print('Recapping the group of interest and reference we defined above: \n\n Attribute: {}\n Group of interest: {}\n Reference Group: {}'.format(attribute_col, group_of_interest, reference_group))

### Let's now try resampling

We can perform three types of resampling:

1. Change the training data such that different poverty levels are distributed more uniformly but keep the distribution of labels the same within each poverty level P(poverty_level = highest) = P (poverty_level=lower)


2. Change the training data such that different poverty levels have more uniform label distributions P(poverty_level = highest | not funded ) = P(poverty_level=lower | not funded)


3. Change both

The following method implements these 3 approaches:

In [ ]:
def sample_groups_equal(df, 
                  attrdf, 
                  label_col, 
                  attribute_col, 
                  group_of_interest, 
                  reference_group, 
                  equal_group_prevalence=True, 
                  equal_group_size=False, 
                  random_seed=42):
    print("Input stats for each group:\n")
    prev_a, size_a, lp_a,ln_a = get_group_stats(df, attrdf, label_col, attribute_col, group_of_interest)
    prev_ref, size_ref, lp_ref,ln_ref = get_group_stats(df, attrdf, label_col, attribute_col, reference_group)
    if equal_group_prevalence:
        # create largest possible sample based on prevalence ratio (minimize number of label positives removed)
        # group_prevalence_ratio = prev_group_of_interest / prev_ref_group
        if prev_a < prev_ref:
            # create subsample
            num_lp_a = len(lp_a)
            target_prev = prev_ref
            new_num_ln_a = math.floor((num_lp_a / target_prev) - num_lp_a)
            ln_a = ln_a.sample(n=new_num_ln_a, replace=False, random_state=random_seed)
        elif prev_ref < prev_a:
            # create subsample
            new_num_lp_ref = len(lp_ref)
            target_prev = prev_a
            new_num_ln_ref = math.floor((num_lp_ref / target_prev) - num_lp_ref)
            ln_ref = ln_ref.sample(n=new_num_ln_ref, replace=False, random_state=random_seed)
    adf = pd.concat([lp_a, ln_a], axis=0)
    refdf = pd.concat([lp_ref, ln_ref], axis=0)
    size_a = len(adf)
    size_ref = len(refdf)
    if equal_group_size:
        # create largest possible sample based on size ratio
        if size_a > size_ref:
            adf = adf.sample(n=size_ref, replace=False, random_state=random_seed)
        elif size_ref > size_a:
            refdf = refdf.sample(n=size_a, replace=False, random_state=random_seed)
    new_traindf = pd.concat([adf, refdf], axis=0)
    print("\n\nNew sample stats for each group:\n")
    _ = get_group_stats(new_traindf, attrdf, label_col, attribute_col, group_of_interest)
    _ = get_group_stats(new_traindf, attrdf, label_col, attribute_col, reference_group)
    return new_traindf

### Let's create 3 new different training dataframes by applying the 3 approaches described above

#### Sampling so we get equal group sizes (randomly undersampling largest group)

In [ ]:
new_traindf_1 = sample_groups_equal(traindf, train_attrdf, label_col, attribute_col, group_of_interest, reference_group, equal_group_prevalence=False, equal_group_size=True)

#### Sampling so we get equal group prevalence in the two groups (undersampling label negatives of group with lower prevalence)

In [ ]:
new_traindf_2 = sample_groups_equal(traindf, train_attrdf, label_col, attribute_col, group_of_interest, reference_group, equal_group_prevalence=True, equal_group_size=False)

#### Sampling so we get both equal group prevalence and group sizes in the two groups 

In [ ]:
new_traindf_3 = sample_groups_equal(traindf, train_attrdf, label_col, attribute_col, group_of_interest, reference_group, equal_group_prevalence=True, equal_group_size=True)

#### Rebuild model for each of the resampled training dataframes and predict on the test set

In [ ]:
def retrain_and_test(model, label_col, new_traindf, testdf):
    y_train = new_traindf[label_col].values
    model.fit(new_traindf.drop(['entity_id','as_of_date',label_col], axis = 1), y_train)
    y_pred = model.predict_proba(testdf.drop(['entity_id','as_of_date',label_col], axis = 1))[:,1]
    new_preds = testdf[['entity_id','as_of_date',label_col]].copy()
    new_preds['predict_proba'] = y_pred
    new_preds = new_preds.sort_values('predict_proba', ascending = False).reset_index(drop=True).copy()
    new_preds['score'] = new_preds.apply(lambda x: 1.0 if int(x.name)  < 1000 else 0.0, axis=1)
    return new_preds

In [ ]:
new_preds_1 = retrain_and_test(rf, label_col, new_traindf_1, testdf)
new_preds_2 = retrain_and_test(rf, label_col, new_traindf_2, testdf)
new_preds_3 = retrain_and_test(rf, label_col, new_traindf_3, testdf)


#### Calculate precision at 1000 for each of the 3 approaches

In [ ]:

print('Approach 1 Model Precision: ', new_preds_1[new_preds_1['score'] > 0][label_col].sum() / 1000)
print('Approach 2 Model Precision: ', new_preds_2[new_preds_2['score'] > 0][label_col].sum() / 1000)
print('Approach 3 Model Precision: ', new_preds_3[new_preds_3['score'] > 0][label_col].sum() / 1000)

## Before and After: Run Aequitas


Let's define an audit util method to use throghout the notebook when audit different models.

In [ ]:
def aequitas_audit(input_df, ref_groups_dict):
    g = Group()
    b = Bias()
    xtab, _ = g.get_crosstabs(input_df[['score','label_value'] + list(ref_groups_dict.keys())].copy())
    bdf = b.get_disparity_predefined_groups(xtab, input_df, ref_groups_dict)
    return bdf


#### Define the configurations for running the bias audits:

    - a dictionary for reference groups in the format attribute:group
    - the metrics of interest
    - the fairness threshold (the ratio of each group value compared to reference group on a given metric)

In [ ]:
ref_groups_dict={'poverty_level':'lower', 'metro_type':'suburban_rural', 'teacher_sex':'male'}
attributes = ref_groups_dict.keys()
metrics = ['tpr']
fairness_threshold = 1.3

For reference, let's start with looking at the "best" model we chose in terms of overall precision at 1,000:

#### Now let's start with loading the predictions from the "best" model chosen earlier

In [ ]:
id_col='entity_id'
date_col = 'as_of_date'
top_k= 1000
old_preds = pd.read_csv(DATAPATH + 'predictions_c598fbe93f4c218ac7d325fb478598f1.csv.gz', compression='gzip')
old_attrdf = pd.read_csv(DATAPATH + 'test_20121201_20130201_protected.csv.gz', compression='gzip')

old_df = pd.merge(old_preds, old_attrdf, how='left', on=[id_col, date_col], sort=True, copy=True)

old_df = old_df.sort_values('predict_proba', ascending=False)
old_df = old_df.rename(columns = {label_col:'label_value'}) # naming for Aequitas

# create a "score" column with the predicted class (named "score" for use with Aequitas below)
old_df['score'] = old_df.apply(lambda x: 1.0 if x.name in old_df.head(top_k).index.tolist() else 0, axis=1)

### Before: Run Aequitas for the "best" model chosen earlier

In [ ]:
bdf_old = aequitas_audit(old_df, ref_groups_dict)

In [ ]:
ap.disparity(bdf_old, metrics, 'poverty_level', fairness_threshold)

### After: Audit for Bias for the new mitigated models  (keeping the attributes, reference groups, bias metric, and tolerance the same as before)

#### Approach 1: Equalizing group sizes

In [ ]:
df_1 = pd.merge(new_preds_1, test_attrdf, how='left', on=['entity_id','as_of_date'], sort=True, copy=True)
df_1 = df_1.rename(columns = {label_col:'label_value'})
bdf_1 = aequitas_audit(df_1, ref_groups_dict)


In [ ]:
ap.disparity(bdf_1, metrics, 'poverty_level', fairness_threshold = 1.3)

In [ ]:
ap.disparity(bdf_1, metrics, 'metro_type', fairness_threshold = 1.3)

In [ ]:
ap.disparity(bdf_1, metrics, 'teacher_sex', fairness_threshold = 1.3)

#### Approach 2: Equalizing group prevalence

In [ ]:
df_2 = pd.merge(new_preds_2, test_attrdf, how='left', on=['entity_id','as_of_date'], sort=True, copy=True)
df_2 = df_2.rename(columns = {label_col:'label_value'})
bdf_2 = aequitas_audit(df_2, ref_groups_dict)


In [ ]:
ap.disparity(bdf_2, metrics, 'poverty_level', fairness_threshold = 1.3)

In [ ]:
ap.disparity(bdf_2, metrics, 'metro_type', fairness_threshold = 1.3)

In [ ]:
ap.disparity(bdf_2, metrics, 'teacher_sex', fairness_threshold = 1.3)

#### Approach 3: Equalizing both group prevalence and group sizes

In [ ]:
df_3 = pd.merge(new_preds_3, test_attrdf, how='left', on=['entity_id','as_of_date'], sort=True, copy=True)
df_3 = df_3.rename(columns = {label_col:'label_value'})
bdf_3 = aequitas_audit(df_3, ref_groups_dict)


In [ ]:
ap.disparity(bdf_3, metrics, 'poverty_level', fairness_threshold = 1.3)

In [ ]:
ap.disparity(bdf_3, metrics, 'metro_type', fairness_threshold = 1.3)

In [ ]:
ap.disparity(bdf_3, metrics, 'teacher_sex', fairness_threshold = 1.3)

### Adding to the model selection (tradeoff) graph

Finally, let's look at how this new option stacks up against what we plotted in our model selection process:

In [ ]:
mitigated_precision_1 = df_1.loc[df_1['score']==1]['label_value'].mean()
mitigated_precision_2 = df_2.loc[df_2['score']==1]['label_value'].mean()
mitigated_precision_3 = df_3.loc[df_3['score']==1]['label_value'].mean()

plot_configs = {
    'evals_df':evals_df, 
    'aequitas_df':aequitas_df,
    'attr_col':'poverty_level', 
    'group_name':'highest',
    'performance_col':'model_precision',
    'bias_metric':'tpr', 
    'flip_disparity':True, 
    'mitigated_tags':['Resampling 1: eq group size','Resampling 3: eq group prev', 'Resampling 3: eq group prev + size'],
    'mitigated_bdfs':[bdf_1, bdf_2, bdf_3],
    'mitigated_performances':[mitigated_precision_1, mitigated_precision_2, mitigated_precision_3],
    'ylim':None
}
create_scatter_disparity_performance(**plot_configs)

## <font color=red>Bias Reduction Strategy 2: Regularization (using FairGBM and the Fairlearn package)</font>
### <font color=red>In-Processing Fairness Improvement</font>

### 1. Import packages and data

In [ ]:
# Let's use the methods 
from fairlearn.reductions import ExponentiatedGradient, GridSearch, DemographicParity, TruePositiveRateDifference
from fairlearn.metrics import selection_rate_group_summary

### What has already happened?

We've already cleaned data, generated features, created train-test sets, built 1000s of models on each training set and scored each test set with them, and calculated various evaluation metrics. We then used these results to pick a "best" model in terms of performance on the "accuracy" metric we care about: **Precision at the top 1000** (corresponding to our goal of selecting 1000 project submissions that are most likely to not get funded in order to prioritize resource allocation).

When we audited this selected model with Aequitas, however, we found biases across many attributes, including the poverty level of the schools. Here, we explore a method of using in-processing to train a fairness-aware classifier in order to reduce this bias.

### <font color=green>FairLearn - a reductions approach</font>

[Paper](https://arxiv.org/pdf/1803.02453.pdf): _A Reductions Approach to Fair Classification_, 2018

> We present a systematic approach for achievingfairness in a binary classification setting. Whilewe focus on two well-known quantitative defini-tions of fairness, our approach encompasses manyother  previously  studied  definitions  as  specialcases. The key idea is to __reduce fair classification__ to a __sequence  of  cost-sensitive__  classification problems, whose solutions yield a randomized classifier with the __lowest (empirical) error__ subject to  the  __desired  constraints__.   We  introduce  two reductions that work for any representation of the cost-sensitive  classifier  and  compare  favorably to prior baselines on a variety of data sets, while overcoming several of their disadvantages.

[FairLearn Documentation](https://fairlearn.github.io/user_guide/mitigation.html#id17)



### TLDR; 

- This approach poses Fair Learning as a constrained optimization problem: minimize the empirical error, subject to linear constraints of the fairness (e.g., TPR difference, demographic parity).
- Solve the constrained optimization as a __cost-sensitive__ classification problem.
- Obtain a __randomized classifier__, which implies they will create multiple base estimators.


### Load train and test matrices as well as protected attributes

In [ ]:
traindf = pd.read_csv(DATAPATH + 'train_20120501_20120801.csv.gz', compression='gzip')
testdf = pd.read_csv(DATAPATH + 'test_20121201_20130201.csv.gz', compression='gzip')
train_attrdf = pd.read_csv(DATAPATH + 'train_20120501_20120801_protected.csv.gz', compression='gzip')
test_attrdf = pd.read_csv(DATAPATH + 'test_20121201_20130201_protected.csv.gz', compression='gzip')

### Set up some parameters we'll need below and create matrices

In [ ]:
label_col = 'quickstart_label'
date_col = 'as_of_date'
id_col = 'entity_id'
attr_col = 'poverty_level'
exclude_cols = [label_col, date_col, id_col]

top_k = 1000

# aequitas parameters
metrics = ['tpr']
disparity_threshold = 1.3
protected_attribute_ref_group = {attr_col:'lower'}


X_train, y_train, A_train = traindf[[c for c in traindf.columns if c not in exclude_cols]].values, traindf[label_col].values, train_attrdf[[attr_col]]
X_test,   y_test,   A_test   = testdf[[c for c in testdf.columns if c not in exclude_cols]].values,   testdf[label_col].values  , test_attrdf[[attr_col]]


### Setting up a fairness-improving classifier

To account fairness during model training, we'll use the **Exponentiated Gradient** provided by the `fairlearn` module.


Its hyperparameters are: 
- `estimator`: an estimator that implements the methods `fit(X, y, sample_weight)` and `predict(X)`.
- `constraints`: fairness constraints.
- `eps: float`: fairness threshold, i.e., how much constraint violation we support (defaults to 0.01). 
- `T: int`: maximum number of iterations (defaults to 50).
- `nu: float`: convergence threshold for duality gap (defaults to None).
- `eta_0: float`: initial learning rate (defaults to 2).
- `run_linprog_step: bool`: whether to apply saddle point optimization to the convex hull of classifiers obtained so far, after each exponentiated gradient step (defaults to True).

In [ ]:
# NOTE: Exponentiated Gradient has a stoachastic component
np.random.seed(0)

Notice that we're using `TruePositiveRateDifference` for our fairness constraint here since we care about equalizing **recall** (aka **tpr** aka **equality of opportunity**) across our subgroups:

In [ ]:
# Step 1. Define the constraint
constraint = TruePositiveRateDifference()

# Step 2. Define the base estimator (any estimator providing 'fit' and 'predict')
# Note: we could have used other algorithm such as logistic regression or random forest
base_estimator = DecisionTreeClassifier(max_depth=20, min_samples_leaf=10)

# Step 3. Define the bias reducer algorithm you want to apply
bias_reducer = ExponentiatedGradient(base_estimator, constraint, T=50)

# Step 4. Fit the data (and provide the sensitive attributes)
bias_reducer.fit(X_train, y_train, sensitive_features=A_train)

### Predict on our test set

In [ ]:
# Step 5. Use the mitigator to make predictions 
y_pred_4 = bias_reducer.predict(X_test)
new_preds_4 = testdf[['entity_id','as_of_date','quickstart_label']].copy()
new_preds_4['score'] = y_pred_4

### Look at the output
Notice that unlike many classifiers, the `ExponentiatedGradient` doesn't have a method for predicting a continuous score, just predicted classes of 0 or 1. How many projects did this model predict are at risk of going unfunded (that is, predicted class of 1)?

In [ ]:
new_preds_4['score'].value_counts()

It looks like about 6,500 projects are predicted as being at risk of going unfunded by this classifier, but unfortunately our program is resource-constrained and can only help 1,000 of them. At this point, if we wanted to pick out the 1,000 highest-risk projects (subject to our fairness constraint), we're a bit stuck: **the classifier doesn't give us any method for distinguishing higher-risk vs lower-risk projects!**

Given this limitation, we might posit that a reasonable approach would be to pick 1,000 projects to intervene with from among these 6,500. Let's see what would happen if we did that...

### Before and After: Run Aequitas

For reference, let's start with looking at the "best" model we chose in terms of overall precision at 1,000:

#### Load the predictions from the "best" model chosen earlier

In [ ]:
old_preds = pd.read_csv(DATAPATH + 'predictions_c598fbe93f4c218ac7d325fb478598f1.csv.gz', compression='gzip')
old_attrdf = pd.read_csv(DATAPATH + 'test_20121201_20130201_protected.csv.gz', compression='gzip')

old_df = pd.merge(old_preds, old_attrdf, how='left', on=[id_col, date_col], sort=True, copy=True)

old_df = old_df.sort_values('predict_proba', ascending=False)
old_df = old_df.rename(columns = {label_col:'label_value'}) # naming for Aequitas

# create a "score" column with the predicted class (named "score" for use with Aequitas below)
old_df['score'] = old_df.apply(lambda x: 1.0 if x.name in old_df.head(top_k).index.tolist() else 0, axis=1)

#### Before: Run Aequitas for the "best" model chosen earlier

In [ ]:
def aequitas_audit(input_df, ref_groups_dict):
    g = Group()
    b = Bias()
    xtab, _ = g.get_crosstabs(input_df[['score','label_value'] + list(ref_groups_dict.keys())].copy())
    bdf = b.get_disparity_predefined_groups(xtab, input_df, ref_groups_dict)
    return bdf


In [ ]:
bdf_old = aequitas_audit(old_df, protected_attribute_ref_group)

ap.disparity(bdf_old, metrics, attr_col, fairness_threshold=disparity_threshold)

#### After: Run Aequitas for the new, fairness-aware model

Remember here that we would choose 1,000 at random from the 6,500 with predicted class 1, so the expected value for the recall disparity of this randomly-selected set would just be the value of full set (that is, the recall of each subgroup would, on average, be proportionally lower in the sub-sample):

In [ ]:
df_4 = pd.merge(new_preds_4, test_attrdf, how='left', on=['entity_id','as_of_date'], sort=True, copy=True)
df_4 = df_4.rename(columns = {label_col:'label_value'})

bdf_4 = aequitas_audit(df_4, protected_attribute_ref_group)

ap.disparity(bdf_4, metrics, attr_col, fairness_threshold=disparity_threshold)

That looks pretty good! The new model appears to have reduced the disparity across poverty levels considerably relative to what we saw when training a model without a fairness constraint and choosing based on precision alone.

However, the natural question here is where there is a fairness-accuracy trade-off here: What cost did we incur in terms of model performance, that is overall precision?

#### Precision of the "best" model chosen earlier

In [ ]:
old_df.loc[old_df['score']==1]['label_value'].mean()

#### Precision of the new, fairness-aware model

As above, we would be sampling from the 6,500 down to 1,000 but the expected value of precision in this sample would just be the mean label value in the full population:

In [ ]:
df_4.loc[df_4['score']==1]['label_value'].mean()

So, compared to the old model, this method has resulted in **quite a large trade-off in model performance to acheive fairness**. This is certainly in part a result of the lack of flexibility in the method not allowing us to provide a score threshold or top k size that we're interested in equalizing our fairness metric around and instead using a built-in threshold that yields 6,500 predicted positives when we're only able to intervene on 1,000. Unfortunately, this sort of inflexibility appears to be a common attribute of many in-processing methods available today.

For context, the overall base rate (`df['label_value'].mean()`) is 0.338, so the drop-off in precision here is about half way from our previous model to simply choosing at random.

### Adding to model selection

Finally, let's look at how this new option stacks up against what we plotted in our model selection process:

In [ ]:
mitigated_precision_4 = df_4.loc[df_4['score']==1]['label_value'].mean()
plot_configs = {
    'evals_df':evals_df, 
    'aequitas_df':aequitas_df,
    'attr_col':'poverty_level', 
    'group_name':'highest',
    'performance_col':'model_precision',
    'bias_metric':'tpr', 
    'flip_disparity':True, 
    'mitigated_tags':['Fairlearn Reductions'],
    'mitigated_bdfs':[bdf_4], 
    'mitigated_performances':[mitigated_precision_4], 
    'ylim':None
}
create_scatter_disparity_performance(**plot_configs)

### Using FairGBM + Hyperparameter optimization

In the following cells we will setup hyperparameter optimization, with FairGBM models.

FairGBM is an algorithm based on LightGBM with fairness constraints applied to the loss function. It was presented earlier this year in ICLR. [link](https://openreview.net/pdf?id=x-mXzBgCX3a)

### <font color=green>FairGBM - Gradient Boosting with Fairness Constraints</font>

> Tabular data is prevalent in many high-stakes domains, such as financial services or public policy. Gradient Boosted Decision Trees (GBDT) are popular in these settings due to their scalability, performance, and low training cost. While fairness in these domains is a foremost concern, existing in-processing Fair ML methods are either incompatible with GBDT, or incur in significant performance losses while taking considerably longer to train. We present FairGBM, a dual ascent learning framework for training GBDT under fairness constraints, with little to no impact on predictive performance when compared to unconstrained GBDT. Since observational fairness metrics are non-differentiable, we propose smooth convex error rate proxies for common fairness criteria, enabling gradient-based optimization using a ``proxy-Lagrangian'' formulation. Our implementation shows an order of magnitude speedup in training time relative to related work, a pivotal aspect to foster the widespread adoption of FairGBM by real-world practitioners.

### TLDR; 

- Similar to Fairlearn, this approach poses Fair Learning as a constrained optimization problem.
- The method uses the algorithm of boosting to adjust the errors of certain instances depending on the constraints.
- We obtain a single final GBM classifier. 


In [ ]:
HYPERPARAM_SPACE = {
    "FairGBM": {
        "classpath": "fairgbm.FairGBMClassifier",
        "kwargs": {
            "boosting_type": ["dart", "gbdt"],
            "multiplier_learning_rate": {
                "type": "float",
                "range": [0.01, 1.0],
                "log": True,
                },
            "proxy_margin": {
                    "type": "float",
                    "range": [0.01, 1.0],
                    "log": True,
                    },
            "constraint_type": "tpr",
            "n_estimators": {
                    "type": "int",
                    "range": [10, 200],
                    "log": True,
                    },
            "num_leaves": {
                    "type": "int",
                    "range": [10, 1000],
                    "log": True,
                    },
            "min_child_samples": {
                    "type": "int",
                    "range": [1, 100],
                    "log": True,
                    },
            "max_depth": {
                    "type": "int",
                    "range": [5, 40],
                    "log": False,
                    },
            "learning_rate": {
                    "type": "float",
                    "range": [0.001, 1.0],
                    "log": True,
                    },
            "global_constraint_type": "fpr,fnr",
            "global_target_fnr": {
                    "type": "float",
                    "range": [0.05, 0.25],
                    "log": False,
                    },
            "global_target_fpr": {
                    "type": "float",
                    "range": [0.02, 0.15],
                    "log": False,
                    },
        }
    }
}

In [ ]:
from hpt.tuner import ObjectiveFunction, OptunaTuner

obj_func = ObjectiveFunction(
    X_train, y_train, X_test, y_test,
    hyperparameter_space=HYPERPARAM_SPACE,    # path to YAML file
    eval_metric="precision",
    other_eval_metric="tpr_ratio",
    s_train=(A_train["poverty_level"]=="highest").astype(int).values,
    s_val=(A_test["poverty_level"]=="highest").astype(int).values,
    ppr=1000/17677,
)

In [ ]:
tuner = OptunaTuner(
    objective_function=obj_func,
    direction="maximize",    # NOTE: can pass other useful study kwargs here (e.g. storage)
)

In [ ]:
tuner.optimize(n_trials=50, n_jobs=1)

In [ ]:
bias_reducer = obj_func.reconstruct_model(obj_func.best_trial)

In [ ]:
y_pred_5 = bias_reducer.predict_proba(X_test)
new_preds_5 = testdf[['entity_id','as_of_date','quickstart_label']].copy()
new_preds_5['score'] = y_pred_5[:, 1]
threshold = new_preds_5['score'].sort_values().values[-1000]
new_preds_5['score'] = (new_preds_5['score'] >= threshold).astype(int)

In [ ]:
df_5 = pd.merge(new_preds_5, test_attrdf, how='left', on=['entity_id','as_of_date'], sort=True, copy=True)
df_5 = df_5.rename(columns = {label_col:'label_value'})

bdf_5 = aequitas_audit(df_5, protected_attribute_ref_group)

ap.disparity(bdf_5, metrics, attr_col, fairness_threshold=disparity_threshold)

In [ ]:
df_5.loc[df_5['score']==1]['label_value'].mean()

## <font color=red>Bias Reduction Strategy 3: Post-Hoc Disparity Mitigation</font>

### What has already happened?

We've already cleaned data, generated features, created train-test sets, built 1000s of models on each training set and scored each test set with them, and calculated various evaluation metrics. We then used these results to pick a "best" model in terms of performance on the "accuracy" metric we care about: **Precision at the top 1000** (corresponding to our goal of selecting 1000 project submissions that are most likely to not get funded in order to prioritize resource allocation).

When we audited this selected model with Aequitas, however, we found biases across many attributes, including the poverty level of the schools. Here, we explore a method of using post-hoc disparity mitigation to reduce this bias in the selected model.

### <font color=green>Intro to Post-Hoc Bias Mitigation</font>

![Diagram of Post-Hoc Adjustments](post_hoc_adj.png "Post-Hoc Adjustments")

One approach to improving the fairness of our model is to make post-hoc adjustments to the thresholds used for each subgroup to choose the 1,000 projects on which to intervene. Because our fairness metric here (**recall** aka **tpr** aka **equality of opportunity**) is monatonically increasing with the depth of the score, we should be able to find score thresholds for each subgroup that will equalize this metric across the groups, subject to the constraint that we want to choose a total of 1,000 projects for our intervation.

In short, here's how this will work (see the references below for a more detailed discussion):
1. Train the model as usual on a training set, predict scores on a test set
2. Split this test set by subgroups on our protected attribute (here, poverty level)
3. Sort each subgroup by score and calculate the cumulative tpr/recall up to and including each row in the set, storing this "rolling within-subgroup recall" value
4. Recombine the subgroups, and sort the entire set by this new value
5. Take the top 1,000 projects from this re-ordered list and use it to calculate "top k" sizes for each subgroup that equalize recall
6. Then, on a future test set, use these calculated subgroup list sizes to assess the impact of disparities and overall precision

References:
- Hardt, et al, [Equality of Opportunity in Supervised Learning](http://papers.nips.cc/paper/6373-equality-of-opportunity-in-supervised-learning)
- Rodolfa, et al, [Case Study: Predictive Fairness to Reduce Misdemeanor Recidivism Through Social Service Interventions](https://dl.acm.org/doi/abs/10.1145/3351095.3372863?casa_token=zc196JJrqkkAAAAA:bPmqmKrA91esJhIxHPT4K1crWWb5JGcflVFDkTgODctMzLpUX50_56Kyyh4NJ2GTd_QSydqhNpjT)

### Load the train, test, and protected attributes from the first split

In [ ]:
split1_traindf = pd.read_csv(DATAPATH + 'train_20111101_20120201.csv.gz', compression='gzip')
split1_testdf = pd.read_csv(DATAPATH + 'test_20120601_20120801.csv.gz', compression='gzip')
split1_attrdf = pd.read_csv(DATAPATH + 'test_20120601_20120801_protected.csv.gz', compression='gzip')

Let's just take a quick look at the data to make sure it makes sense:

In [ ]:
split1_traindf.head()

### Set up some parameters we'll need below

Note that the classifier type and hyperparameters here are from the best-performing model we chose above based on precision on the top 1000.

In [ ]:
hyperparameters = {
    'criterion': 'gini',
    'max_depth': 30,
    'max_features': 'sqrt',
    'min_samples_leaf': 44,
    'min_samples_split': 3,
    'n_estimators': 87,
    'n_jobs': -1,
    'random_state': 213500298
}
clf = RandomForestClassifier(**hyperparameters)

top_k = 1000

label_col = 'quickstart_label'
entity_col = 'entity_id'
date_col = 'as_of_date'
exclude_cols = [label_col, entity_col, date_col] # columns to exclude from the X matrices for the classifier

protected_attribute_col = 'poverty_level'

# Parameters for Aequitas

metrics = ['tpr']
disparity_threshold = 1.3
protected_attribute_ref_group = {protected_attribute_col:'lower'}


### Train the model and predict on the test set

In [ ]:
# train
y_train = split1_traindf[label_col].values
clf.fit(split1_traindf.drop(exclude_cols, axis = 1), y_train)

# test set predictions
split1_preds = split1_testdf[[entity_col, date_col, label_col]].copy()
split1_preds['predict_proba'] = clf.predict_proba(split1_testdf.drop(exclude_cols, axis = 1))[:,1]

Let's take a quick look at the predictions to make sure they look good:

In [ ]:
split1_preds.head()

### Combine predictions with protected attributes

In [ ]:
df = pd.merge(split1_preds, split1_attrdf, how='left', on=[entity_col,date_col], sort=True, copy=True)

### Sort by score, then split by protected attribute (poverty level)

In [ ]:
protected_attribute_groups = df[protected_attribute_col].unique()

In [ ]:
df = df.sort_values('predict_proba', ascending=False)
subgroup_dfs = []
for grp in protected_attribute_groups:
    subgroup_dfs.append(df[df[protected_attribute_col]==grp].copy())

### Calculate within-subgroup cumulative recall

Here, we calculate the recall up to and including each row within the highest-poverty and lower-poverty subsets of the test set. Doing this allows us to recombine and sort the sets in a way that will let find recall-equalizing "top k" list sizes for each subgroup.

In [ ]:
for subgrp_df in subgroup_dfs:
    subgrp_df['cumsum_recall'] = subgrp_df[label_col].cumsum() / subgrp_df[label_col].sum()

### Recombine the subgroup sets and sort by this cumulative recall

In [ ]:
recall_df = pd.concat(subgroup_dfs, axis=0).sort_values('cumsum_recall', ascending=True)

### Find subgroup sizes, holding the overall list size (1000) constant

Now we can simply threshold this re-sorted list by `top_k` (here, 1000) to identify how many individuals from each group we should apply in the future.

Notice here that each subgroup will still be ordered by their predicted score, but the scores will no longer be perfectly ordered across subgroups.

In [ ]:
new_pp = recall_df.head(top_k).copy()
new_pp.tail()

In [ ]:
new_pp[protected_attribute_col].value_counts()

Let's just store this to re-use below:

In [ ]:
subgroup_k = {} 
for grp in protected_attribute_groups:
  subgroup_k[grp] = new_pp[protected_attribute_col].value_counts()[grp]

### Apply these subgroup-specific sizes to future test set data

Now we have calculated the number of projects we need to select from each poverty level, we can apply these to the most recent split to assess how well this method reduces the recall disparities we saw initially and whether this has any impact on the overall precision of the model.

### Load the predictions and protected attributes from the future test set

Note that the predictions here correspond to the same model + hyperparameters we specified above

In [ ]:
split2_preds = pd.read_csv(DATAPATH + 'predictions_c598fbe93f4c218ac7d325fb478598f1.csv.gz', compression='gzip')
split2_attrdf = pd.read_csv(DATAPATH + 'test_20121201_20130201_protected.csv.gz', compression='gzip')

df2 = pd.merge(split2_preds, split2_attrdf, how='left', on=[entity_col, date_col], sort=True, copy=True)

Take a quick look to make sure the data loaded without any issue:

In [ ]:
df2.head()

### Split the test set by poverty level to apply the thresholds

In [ ]:
df2 = df2.sort_values('predict_proba', ascending=False)
new_subgroup_dfs = {}
for grp in protected_attribute_groups:
    new_subgroup_dfs[grp] = df2[df2[protected_attribute_col]==grp].copy()

### Choose the number of projects from each subgroup found above

Notice here that we're choosing the "top k" individuals within each subgroup based on their predicted score -- in a deployment, we wouldn't know the true labels to calculate recall values, which is why we had to go one step back in time to find these group sizes.

In [ ]:
pp_dfs = []
for grp in protected_attribute_groups:
    pp_dfs.append(new_subgroup_dfs[grp].head(subgroup_k[grp]))

### Recombine and create a predicted class label for this resulting set
That is, 1,000 projects with a label 1 chosen by this process and 0 otherwise

In [ ]:
new_pp2 = pd.concat(pp_dfs, axis=0).sort_values('predict_proba', ascending=True)

mitigated_df = df2.copy()
mitigated_df = mitigated_df.rename(columns = {label_col:'label_value'}) # naming for Aequitas

# create a "score" column with the predicted class (named "score" for use with Aequitas below)
mitigated_df['score'] = mitigated_df.apply(lambda x: 1.0 if x.name in new_pp2.index.tolist() else 0, axis=1)

for comparison, let's also look at the unmitigated result again...

In [ ]:
unadjusted_df = df2.sort_values('predict_proba', ascending=False).copy()
unadjusted_df = unadjusted_df.rename(columns = {label_col:'label_value'}) # naming for Aequitas

# create a "score" column with the predicted class (named "score" for use with Aequitas below)
unadjusted_df['score'] = unadjusted_df.apply(lambda x: 1.0 if x.name in unadjusted_df.head(top_k).index.tolist() else 0, axis=1)

### Running Aequitas - Before and After

#### For the original score, without post-hoc adjustment for equity

In [ ]:
bdf_unadjusted = aequitas_audit(unadjusted_df, protected_attribute_ref_group)
ap.disparity(bdf_unadjusted, metrics, protected_attribute_col, fairness_threshold=disparity_threshold)

#### For the score, with post-hoc disparity mitigation

In [ ]:
bdf_mitigated = aequitas_audit(mitigated_df, protected_attribute_ref_group)

ap.disparity(bdf_mitigated, metrics, protected_attribute_col, fairness_threshold=disparity_threshold)

So, it looks the post-hoc adjustments have actually manage to mitigate the existing disparity pretty well (perhaps even over-shooting somewhat, though still within our fairness threshold of 1.3).

However, the natural question here is where there is a fairness-accuracy trade-off here: What cost did we incur in terms of model performance, that is overall precision?

#### Before: Precision of the original, unadjusted score

In [ ]:
unadjusted_df.loc[unadjusted_df['score']==1]['label_value'].mean()

#### After: Precision of the new, disparity-mitigated score

In [ ]:
mitigated_df.loc[mitigated_df['score']==1]['label_value'].mean()

Somewhat surprisingly, we actually don't seem to see any trade-off with the disparity mitigation here!

### Adding to the model selection (tradeoff) graph

Finally, let's look at how this new option stacks up against what we plotted in our model selection process:

In [ ]:
mitigated_precision_5 = mitigated_df.loc[mitigated_df['score']==1]['label_value'].mean()
plot_configs = {
    'evals_df':evals_df, 
    'aequitas_df':aequitas_df,
    'attr_col':'poverty_level', 
    'group_name':'highest',
    'performance_col':'model_precision',
    'bias_metric':'tpr', 
    'flip_disparity':True, 
    'mitigated_tags':['Equalizing Recall'],
    'mitigated_bdfs':[bdf_mitigated], 
    'mitigated_performances':[mitigated_precision_5], 
    'ylim':None
}
create_scatter_disparity_performance(**plot_configs)

# Wrapping-up

## Overview of all mitigation strategies on the model selection (tradeoff) graph


In [ ]:
plot_configs = {
    'evals_df':evals_df, 
    'aequitas_df':aequitas_df,
    'attr_col':'poverty_level', 
    'group_name':'highest',
    'performance_col':'model_precision',
    'bias_metric':'tpr', 
    'flip_disparity':True, 
    'mitigated_tags':['Resampling 1: eq group size','Resampling 3: eq group prev', 'Resampling 3: eq group prev + size','Fairlearn Reductions','Equalizing Recall'],
    'mitigated_bdfs':[bdf_1, bdf_2, bdf_3, bdf_4, bdf_mitigated], 
    'mitigated_performances':[mitigated_precision_1, mitigated_precision_2, mitigated_precision_3,mitigated_precision_4, mitigated_precision_5], 
    'ylim':None
}
create_scatter_disparity_performance(**plot_configs)